In [1]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
%matplotlib inline
### imports ###
import sys
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns

matplotlib.rcParams['savefig.dpi'] = 200
sys.setrecursionlimit(3000)
sns.set(font_scale=1)
sns.set_context('talk')
sns.set_style('white')

# import custom functions
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from homer_preprocessing import read_homer_gene_exp, import_homer_diffgene, pull_comparisons_get_diff
from plotting_scripts import label_point, pca_rpkm_mat, get_diff_volcano, plot_exp_rpkm, gene_list_bar_plot

In [2]:
workingDirectory = '/home/h1bennet/strains_rna/results/WL01_Strains_AMLN_21week_WholeLiver_RNA/'
dataDirectory = '/home/h1bennet/strains/data/RNA/AMLN_21week_wholeliver/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)


# Check qc stats

In [5]:
qc = ngs_qc(dataDirectory)

/home/h1bennet/strains/data/RNA/AMLN_21week_wholeliver//
./AMLN_21week_wholeliver_qc/


<Figure size 432x288 with 0 Axes>

In [6]:
qc

,uniquePositions,fragmentLengthEstimate,tagsPerBP,clonality,GC_Content,totalReads,uniquelyMappedReads,multiMappedReads,frac_unmappedReads_mismatch,frac_unmappedReads_short,frac_unmappedReads_other,uniquelyMappedFraction,mappedFraction
NCoRWT_WholeLiver_RNA_AMLNDiet_21week_LN170A_161019,6107604.0,75.0,0.012111,5.396,0.512,37945243.0,32956416.0,4137482.0,0.0,0.0038,0.0040,0.868526,0.9922
NCoRWT_WholeLiver_RNA_AMLNDiet_21week_LN170B_161019,6948606.0,75.0,0.011276,4.414,0.518,35329496.0,30684128.0,3826586.0,0.0,0.0045,0.0037,0.868513,0.9918
NCoRWT_WholeLiver_RNA_AMLNDiet_21week_LN170C_161019,5595523.0,75.0,0.010015,4.870,0.528,31120429.0,27250633.0,3241967.0,0.0,0.0050,0.0039,0.875651,0.9911
aj_WholeLiver_RNA_AMLNDiet_21week_AJ8A_160829.aj.star_shifted_from_AJ.sam,5626634.0,75.0,0.010661,5.152,-1.000,33030743.0,29004994.0,3422846.0,0.0,0.0038,0.0035,0.878121,0.9927
aj_WholeLiver_RNA_AMLNDiet_21week_AJ8C_160915.aj.star_shifted_from_AJ.sam,1444776.0,75.0,0.008585,16.163,-1.000,26769361.0,23351249.0,2827567.0,0.0,0.0045,0.0035,0.872313,0.9920
aj_WholeLiver_RNA_AMLNDiet_21week_AJ8D_160915.aj.star_shifted_from_AJ.sam,5010524.0,75.0,0.009026,4.901,-1.000,28138274.0,24556200.0,2964593.0,0.0,0.0047,0.0035,0.872697,0.9918
balbc_WholeLiver_RNA_AMLNDiet_21week_BALB8A_160915.balbcj.star_shifted_from_BALBCJ.sam,5807077.0,75.0,0.008517,3.992,-1.000,26717749.0,23180614.0,3172059.0,0.0,0.0037,0.0040,0.867611,0.9923
balbc_WholeLiver_RNA_AMLNDiet_21week_BALB8B_160829.balbcj.star_shifted_from_BALBCJ.sam,3074358.0,75.0,0.006315,5.589,-1.000,19444021.0,17182812.0,1989263.0,0.0,0.0060,0.0032,0.883707,0.9908
balbc_WholeLiver_RNA_AMLNDiet_21week_BALB8D_160915.balbcj.star_shifted_from_BALBCJ.sam,5600169.0,75.0,0.007812,3.796,-1.000,24152887.0,21257409.0,2596098.0,0.0,0.0037,0.0035,0.880119,0.9928


# Run Oyoung's pipeline to process RNA-seq data consistantly.

In [9]:
c57_paths = list(np.sort(glob.glob(dataDirectory+'/NCoRWT*')))
balbc_paths = list(np.sort(glob.glob(dataDirectory+'/balb*')))
aj_paths = list(np.sort(glob.glob(dataDirectory+'/aj*')))

In [10]:
c57_ids = [i.split('/')[-1].split('_')[5] for i in c57_paths]
balbc_ids = [i.split('/')[-1].split('_')[5] for i in balbc_paths]
aj_ids = [i.split('/')[-1].split('_')[5] for i in aj_paths]

In [11]:
ids = ['c57_wholeliver_amlndiet_21weeks',
       'balbc_wholeliver_amlndiet_21weeks',
       'aj_wholeliver_amlndiet_21weeks']
colors = ['#006d2c',
          '#08519c',
          '#a50f15']
paths = [';'.join(c57_paths),
         ';'.join(balbc_paths),
         ';'.join(aj_paths)]
short_ids = [';'.join([ids[0]+'_'+i for i in c57_ids]),
             ';'.join([ids[1]+'_'+i for i in balbc_ids]),
             ';'.join([ids[2]+'_'+i for i in aj_ids])]

In [12]:
pd.DataFrame([ids,colors,paths,short_ids]).T.to_csv('./sampleDef.txt',
                                                    sep='\t',
                                                    header=False,
                                                    index=False)

# Run O'young RNA pipeline

Run the code below in BASH

    source activate r-ouyangPipe
    
    mkdir ./expression/
    
    mkdir ./differential/
    
    rnaQuan.R ./sampleDef.txt -o ./expression/

    rnaDiff.R ./sampleDef.txt -c ./expression/rawC.txt \
    -t ./expression/rawT.txt -o ./differential/

    # annotate scatter files with tag counts
    for scatter in ./differential/*scatter.txt;
    do diff2Homer.R -d $scatter -q ./expression/HOMER.rawTPM.txt;
    done

### Make sure to check the PCA pdf file for outliers

# Import differential gene files for examination and plotting

## Write differential genes to list for metascape

In [13]:
if not os.path.isdir('./gene_lists/'):
    os.mkdir('./gene_lists')

In [14]:
strains = ['aj', 'balb', 'c57']
log2fc = 1
padj = 0.05

In [15]:
gene_list_dict ={}

In [16]:
for strain in strains:
    # initialize gene lists
    g_intersect = set()
    g_union = set()
    for i in glob.glob('./differential/'+strain+'*homer.txt'):
        print(i)
        # read in gene expression data
        df, tpm, tpm_mat, tpm_mat_quant = import_homer_diffgene(
            depath=i,
            gepath='./expression/HOMER.rawTPM.txt',
            gene_name_index=True)

        # select differential genes
        df = df.loc[~(df.chr.str.contains('chrX|chrY|ChrUn')), :]
        sig_genes = df.index[(df['padj'] < padj) & (df['log2FoldChange'] >= log2fc)].to_list()
        
        # add to sets
        if len(g_union) == 0:
            g_union = set(sig_genes)
            g_intersect = set(sig_genes)
        else:
            g_union = g_union.union(sig_genes)
            g_intersect = g_intersect.intersection(sig_genes)
        
        # print('union set length:', print(len(g_union)))
        # print('intersection set length:', print(len(g_intersect)))
    
    # after getting union and intersection sets add to dict
    gene_list_dict[strain+'_union'] = list(g_union)
    gene_list_dict[strain+'_intersection'] = list(g_intersect)
        

./differential/aj_wholeliver_amlndiet_21weeks.vs.balbc_wholeliver_amlndiet_21weeks.scatter.homer.txt
ge all transcripts (24940, 16)
Diff Gene selected transcripts (6012, 14)
ge selected transcripts (6012, 16)
./differential/aj_wholeliver_amlndiet_21weeks.vs.c57_wholeliver_amlndiet_21weeks.scatter.homer.txt
ge all transcripts (24940, 16)
Diff Gene selected transcripts (6228, 14)
ge selected transcripts (6228, 16)
./differential/balbc_wholeliver_amlndiet_21weeks.vs.c57_wholeliver_amlndiet_21weeks.scatter.homer.txt
ge all transcripts (24940, 16)
Diff Gene selected transcripts (6308, 14)
ge selected transcripts (6308, 16)
./differential/balbc_wholeliver_amlndiet_21weeks.vs.aj_wholeliver_amlndiet_21weeks.scatter.homer.txt
ge all transcripts (24940, 16)
Diff Gene selected transcripts (6012, 14)
ge selected transcripts (6012, 16)
./differential/c57_wholeliver_amlndiet_21weeks.vs.balbc_wholeliver_amlndiet_21weeks.scatter.homer.txt
ge all transcripts (24940, 16)
Diff Gene selected transcripts (

In [17]:
for key in gene_list_dict.keys():
    print(key)
    print(len(gene_list_dict[key]), 'genes')
    with open('./gene_lists/'+key+'_wholeliver_amln_genes_fc_'+str(log2fc)+'_pval_'+str(padj)+'.txt', 'w') as f:
        f.write('Gene\n')
        for i in gene_list_dict[key]:
            f.write(i+'\n')
        f.close()


aj_union
242 genes
aj_intersection
33 genes
balb_union
313 genes
balb_intersection
55 genes
c57_union
192 genes
c57_intersection
42 genes
